In [25]:
import paramiko
import ast
import pandas as pd
import logging
import logging.config
from sshtunnel import SSHTunnelForwarder
import pymysql

from sqlalchemy import create_engine, text

In [26]:
USER='postgres'
PASSWORD='A123456*'
HOST='localhost'
PORT='5433'
DATABASE='utea'

In [27]:
# ruta de carpera donde estan las llaves
ruta = open('../unidad_local.txt')
ruta = ruta.readlines()[0]

# cargar datos para conectar a AGROCITTCA
data = open(ruta + '/_keys/agrocittca.data','r')
password = data.readlines()[0]
data.close()

# extraccion de los datos
datos_cnx = ast.literal_eval(password)

# lectura de llave ssh
mypkey = paramiko.RSAKey.from_private_key_file(ruta + '/_keys/llave_db_agrocittca', password='micontrasena')

# ssh variables
ssh_host = datos_cnx['ssh_host']
ssh_port = datos_cnx['ssh_port']
localhost = datos_cnx['localhost']
localport = datos_cnx['localport']
ssh_user = datos_cnx['ssh_user']

# database variables
sql_username = datos_cnx['sql_username']
sql_password = datos_cnx['sql_password']
sql_main_database = datos_cnx['sql_main_database']

In [28]:
# funcion para ejecutar una script SQL
# estado = 0 para select
# estado = 1 para update
def dbQuery(q, estado = 0):
    """ access the database over the SSH tunnel and execute the query """
    logger = logging.getLogger()
    logger.debug("Connecting to reporting server...", end="")
    with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(localhost, 3306)) as tunnel:
        logger.debug("Connected")
        logger.debug("Connecting to database...")
        try:
            conn = pymysql.connect(
                host='127.0.0.1', 
                user=sql_username,
                passwd=sql_password,
                db=sql_main_database,
                port=tunnel.local_bind_port,
                cursorclass=pymysql.cursors.DictCursor)
            logger.debug("Connected")
            logger.debug("Executing query...")
            cursor = conn.cursor()
            cursor.execute(q)
            if estado==1:
                conn.commit()  # Commit the transaction
            data = cursor.fetchall()
            logger.debug("Done")
        finally:
            conn.close()
    return data

In [29]:
def dbQueryInsect(q):
    """Access the database over the SSH tunnel and execute an INSERT/UPDATE query."""
    try:
        with SSHTunnelForwarder(
            (ssh_host, ssh_port),
            ssh_username=ssh_user,
            ssh_pkey=mypkey,
            remote_bind_address=(localhost, 3306)
        ) as tunnel:
            with pymysql.connect(
                host='127.0.0.1',
                user=sql_username,
                passwd=sql_password,
                db=sql_main_database,
                port=tunnel.local_bind_port,
                cursorclass=pymysql.cursors.DictCursor
            ) as conn:
                with conn.cursor() as cursor:
                    cursor.execute(q)
                    conn.commit()
                    return True   # Éxito
    except Exception as e:
        print(f"Error al ejecutar query: {e}")
        return False  # Fallo

def dbQuerySelect(q):
    """Access the database over the SSH tunnel and execute a SELECT query."""
    try:
        with SSHTunnelForwarder(
            (ssh_host, ssh_port),
            ssh_username=ssh_user,
            ssh_pkey=mypkey,
            remote_bind_address=(localhost, 3306)
        ) as tunnel:
            with pymysql.connect(
                host='127.0.0.1',
                user=sql_username,
                passwd=sql_password,
                db=sql_main_database,
                port=tunnel.local_bind_port,
                cursorclass=pymysql.cursors.DictCursor
            ) as conn:
                with conn.cursor() as cursor:
                    cursor.execute(q)
                    data = cursor.fetchall()  # Trae todos los resultados
                    return data
    except Exception as e:
        print(f"Error al ejecutar SELECT: {e}")
        return None  # En caso de error

def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
    )

In [30]:
def update_estado_agrocittca(id_resumen):
    try:
        engine = obtener_engine()
        with engine.begin() as conn:  # begin() hace commit automático si no hay error
            conn.execute(
                text("UPDATE drones_control_bio.resumen_control_bio SET agrocittca = true WHERE id = :id"),
                {"id": id_resumen}
            )
        return True  # Éxito
    except Exception as e:
        print(f"Error al ejecutar update_estado_agrocittca: {e}")
        return False  # Fallo

In [31]:
def get_id_max_informe_caniero():
    query_max = 'SELECT MAX(id) AS maxid FROM informe_caniero'
    try:
        max_id = dbQuerySelect(query_max)
        if not max_id or max_id[0]['maxid'] is None:  
            return None  # No hay registros en la tabla
        return int(max_id[0]['maxid'])
    except Exception as e:
        print(f"Error al obtener MAX(id): {e}")
        return None

In [32]:
df = pd.read_sql(
    "SELECT * FROM drones_control_bio.resumen_control_bio WHERE agrocittca=false", 
    obtener_engine()
)

In [33]:
# concatena codigo y nombre cañero
df['nombre'] = df.apply(lambda x: f"{x['cod_canero']} - {x['nombre_canero']}", axis=1)

# selecciona las columnas deseadas
df = df[['id', 'cod_canero', 'nombre', 'fecha', 'link']]

# agrega las columnas faltantes
df['estado'] = 'activo'
df['informe_enviado'] = 'no'
df['slug'] = ''
df['tipo_informe_caniero_id'] = 5

print(len(df))
df.head(3)

21


,id,cod_canero,nombre,fecha,link,estado,informe_enviado,slug,tipo_informe_caniero_id
0,425,12162,12162 - ROCA REYES MARCOS,2025-12-19,https://drive.google.com/file/d/1MxXDF-jI6txuG...,activo,no,,5
1,432,14993,14993 - VACA CUELLAR JORGE,2025-12-18,https://drive.google.com/file/d/1iEvmdAAvdCYD-...,activo,no,,5
2,435,1530,1530 - C.I.T.T.C.A.,2025-12-10,https://drive.google.com/file/d/1HaZS5tBn-NjSH...,activo,no,,5


In [34]:
max_id = get_id_max_informe_caniero()
max_id

2473

In [35]:
for index, row in df.iterrows():
    id_resumen = row['id']
    max_id = max_id + 1
    sql_insert = f'''INSERT INTO informe_caniero (
                        codigo_caniero, 
                        nombre_caniero,
                        fecha,
                        enlace,
                        estado,
                        informe_enviado,
                        slug, 
                        tipo_informe_caniero_id,
                        created_at, 
                        updated_at) 
                VALUES (
                        '{row['cod_canero']}', 
                        '{row['nombre']}', 
                        '{row['fecha']}', 
                        '{row['link']}', 
                        '{row['estado']}', 
                        '{row['informe_enviado']}', 
                        '{'slug' + '-' + str(max_id)}',
                        {row['tipo_informe_caniero_id']},
                        NOW(),
                        NOW())'''
    registrar_informe = dbQueryInsect(sql_insert)
    if registrar_informe == True:
        print(f"Nuevo dato insertado de {row['nombre']}")
        update_resumen = update_estado_agrocittca(id_resumen)
        if update_resumen == True:
            print(f"Marcado como envaido a AgroCITTCA de {row['nombre']}")
        else:
            print(f"No se pudo marcar como enviado de {row['nombre']}")
    else:
        print(f'''Error al registrar informe de {row['nombre']}''')

Nuevo dato insertado de 12162 - ROCA REYES MARCOS
Marcado como envaido a AgroCITTCA de 12162 - ROCA REYES MARCOS
Nuevo dato insertado de 14993 - VACA CUELLAR JORGE
Marcado como envaido a AgroCITTCA de 14993 - VACA CUELLAR JORGE
Nuevo dato insertado de 1530 - C.I.T.T.C.A.
Marcado como envaido a AgroCITTCA de 1530 - C.I.T.T.C.A.
Nuevo dato insertado de 2029 - COLAMARINO DI SILVIO ANGELO
Marcado como envaido a AgroCITTCA de 2029 - COLAMARINO DI SILVIO ANGELO
Nuevo dato insertado de 2549 - CAMARGO REQUENA MARCO ANTONIO
Marcado como envaido a AgroCITTCA de 2549 - CAMARGO REQUENA MARCO ANTONIO
Nuevo dato insertado de 2662 - DON SERAFIN SRL.
Marcado como envaido a AgroCITTCA de 2662 - DON SERAFIN SRL.
Nuevo dato insertado de 2662 - DON SERAFIN SRL.
Marcado como envaido a AgroCITTCA de 2662 - DON SERAFIN SRL.
Nuevo dato insertado de 40004 - AGUILERA RIVERO NICOLAS JULIO
Marcado como envaido a AgroCITTCA de 40004 - AGUILERA RIVERO NICOLAS JULIO
Nuevo dato insertado de 41594 - AGROPECUARIA CAMPO